### 🎯 Project : Student Data Management System


In [1]:
import csv
import os

# ----- Grade Point Mapping -----
# This dictionary converts letter grades into numerical points for GPA calculation.
GRADE_POINTS = {
    'A+': 4.0, 'A': 4.0,
    'B+': 3.5, 'B': 3.0,
    'C+': 2.5, 'C': 2.0,
    'FAIL': 0.0
}

# --- File paths for data persistence ---
STUDENTS_FILE = 'students.csv'
COURSES_FILE = 'courses.csv'
ENROLLMENTS_FILE = 'enrollments.csv'


# ----- Class Definitions -----

class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age


class Student(Person):
    def __init__(self, name, age, student_id):
        super().__init__(name, age)
        self.student_id = student_id
    
    def get_status(self):
        return "Undergraduate"
    
    def __str__(self):
        return f"Student Name : {self.name}, Student's Age : {self.age}, Student ID : {self.student_id}"

    def calculate_course_grade(self, course):
        """
        Calculates a UG student's grade by summing up marks from each subject.
        Returns a tuple: (final_percentage, final_grade).
        """
        print(f"\n--- Grading Undergraduate Student for {course.course_name} ---")
        if not course.subjects:
            print(f"Error: Course '{course.course_name}' has no subjects. Cannot calculate grade.")
            return None, None

        total_obtained_marks = 0
        total_possible_marks = len(course.subjects) * 100  # Assuming each subject is out of 100

        for subject in course.subjects:
            while True:
                try:
                    marks = int(input(f"  Enter marks for '{subject.subject_name}' (0-100): "))
                    if 0 <= marks <= 100:
                        total_obtained_marks += marks
                        break
                    else:
                        print("  Marks must be between 0 and 100.")
                except ValueError:
                    print("  Invalid input. Please enter a number.")
        
        final_percentage = (total_obtained_marks / total_possible_marks) * 100
        
        # Determine letter grade from the final percentage
        if final_percentage >= 90: grade = 'A+'
        elif final_percentage >= 80: grade = 'A'
        elif final_percentage >= 70: grade = 'B+'
        elif final_percentage >= 60: grade = 'B'
        elif final_percentage >= 50: grade = 'C+'
        elif final_percentage >= 33: grade = 'C'
        else: grade = 'FAIL'
        
        return f"{final_percentage:.2f}", grade
    
    def calculate_gpa(self, enrollment_data, courses_list):
        """Calculates the Grade Point Average (GPA) for an undergraduate student."""
        total_points = 0
        total_credits = 0

        # Check if the student is enrolled in any courses
        if self.student_id not in enrollment_data:
            return 0.0

        # Loop through each enrolled course to calculate points
        student_enrollments = enrollment_data[self.student_id]
        for course_code, details in student_enrollments.items():
            grade = details['grade']
            
            # Only include courses that have been graded
            if grade in GRADE_POINTS:
                # Find the course to get its credit value
                course_credit = 0
                for c in courses_list:
                    if c.course_code == course_code:
                        course_credit = c.course_credit
                        break
                
                # Calculate points for this course and add to totals
                grade_point = GRADE_POINTS[grade]
                total_points += grade_point * course_credit
                total_credits += course_credit
        
        # Avoid division by zero if no courses have credits or grades
        if total_credits == 0:
            return 0.0
        
        return total_points / total_credits

class GraduateStudent(Student):
    def __init__(self, name, age, student_id, thesis_topic):
        super().__init__(name, age, student_id)
        self.thesis_topic = thesis_topic
        self.thesis_grade = "Not Graded"

    def get_status(self):
        return "Graduate"
    
    def __str__(self):
        return f"{super().__str__()}, Thesis: {self.thesis_topic}"
    
    def calculate_course_grade(self, course):
        """
        Calculates a Graduate student's grade for a course, and then
        optionally asks to grade their thesis at the same time.
        """
        print(f"\n--- Grading Graduate Student for {course.course_name} ---")
        
        # --- PART 1: Grade the Course (Same as before) ---
        marks = -1
        while not (0 <= marks <= 100):
            try:
                marks = int(input(f"  Enter overall marks for course '{course.course_name}' (0-100): "))
            except ValueError:
                print("  Invalid input. Please enter a number.")
        
        grade = 'A+' if marks >= 90 else 'A' if marks >= 80 else 'B+' if marks >= 70 else 'B' if marks >= 60 else 'C+' if marks >= 50 else 'C' if marks >= 33 else 'FAIL'
        
        # --- PART 2: Optionally Grade the Thesis (NEW) ---
        if input("\nDo you also want to grade this student's thesis now? (y/n): ").lower() == 'y':
            while True:
                thesis_grade_input = input(f"  Enter thesis grade for {self.name} (e.g., A+, B, etc.): ").upper()
                if thesis_grade_input in GRADE_POINTS:
                    self.thesis_grade = thesis_grade_input
                    print(f"  ✅ Thesis grade '{self.thesis_grade}' has been recorded.")
                    break
                else:
                    print(f"  Error: '{thesis_grade_input}' is not a valid grade. Please try again.")

        return str(marks), grade
    
    def calculate_gpa(self, enrollment_data, courses_list):
        """
        Calculates GPA for a graduate student.
        - Rule: Grades below 'B' are excluded.
        - Includes the graded thesis, weighted as 6 credits.
        """
        total_points = 0
        total_credits = 0
        THESIS_CREDITS = 6  # Define the credit weight of a thesis

        # --- Part 1: Calculate GPA from regular courses ---
        if self.student_id in enrollment_data:
            student_enrollments = enrollment_data[self.student_id]
            for course_code, details in student_enrollments.items():
                grade = details['grade']
                # Rule: Exclude grades below 'B'
                if grade in GRADE_POINTS and grade not in ['C+', 'C', 'FAIL']:
                    course_credit = 0
                    for c in courses_list:
                        if c.course_code == course_code:
                            course_credit = c.course_credit
                            break
                    grade_point = GRADE_POINTS[grade]
                    total_points += grade_point * course_credit
                    total_credits += course_credit
        
        # --- Part 2: Add the thesis grade to the calculation ---
        if self.thesis_grade in GRADE_POINTS:
            # A helpful message will appear on the report card
            print(f"  (Including thesis grade: {self.thesis_grade})")
            thesis_points = GRADE_POINTS[self.thesis_grade] * THESIS_CREDITS
            total_points += thesis_points
            total_credits += THESIS_CREDITS

        # --- Part 3: Final Calculation ---
        if total_credits == 0:
            return 0.0
        
        return total_points / total_credits


class Subject:
    def __init__(self, subject_name, credits):
        self.subject_name = subject_name
        self.credits = credits

    def __str__(self):
        return f"{self.subject_name} ({self.credits} credits)"


class Course:
    def __init__(self, course_name, course_code, course_credit):
        self.course_name = course_name
        self.course_code = course_code
        self.course_credit = course_credit
        self.subjects = []

    def add_subject(self, subject_object):
        current_subject_credits = sum(s.credits for s in self.subjects)
        if current_subject_credits + subject_object.credits > self.course_credit:
            print(f"\nError: Cannot add subject. Exceeds total course credit of {self.course_credit}.")
            return False
        else:
            self.subjects.append(subject_object)
            print(f"\nSuccess: Subject '{subject_object.subject_name}' added to course '{self.course_name}'.")
            return True

    def get_remaining_credits(self):
        return self.course_credit - sum(s.credits for s in self.subjects)

    def __str__(self):
        subject_info = ", ".join([str(s) for s in self.subjects]) or "No subjects added yet."
        return (f"Course Name: {self.course_name}, Code: {self.course_code}, Total Credit: {self.course_credit}"
                f"\n   Subjects: [{subject_info}]")


class Gradebook:
    def __init__(self):
        self.students = []
        self.courses = []
        self.enrollment = {}
        self.load_data() # Automatically load data on startup

    # --- Student Methods ---
    def add_student(self, new_student_object):
        # Prevent duplicate student IDs
        if any(s.student_id == new_student_object.student_id for s in self.students):
            print(f"\nError: Student with ID {new_student_object.student_id} already exists.")
            return
        self.students.append(new_student_object)

    def show_student(self):
        if not self.students:
            print("\nNo Students have been added yet.")
            return
        print("\n" + "-"*65)
        print(f"{'--- Student Roster ---':^65}")
        print("-" * 65)
        print(f"{'Student ID':<15} | {'Name':<25} | {'Age':<5} | {'Status':<15}")
        print("-" * 65)
        for s in self.students:
            print(f"{s.student_id:<15} | {s.name:<25} | {s.age:<5} | {s.get_status():<15}")
        print("-" * 65)


    def show_graduate_students(self):
        """Filters and displays only the graduate students in a formatted table."""
        # Use a list comprehension to find all instances of GraduateStudent
        grad_students = [s for s in self.students if isinstance(s, GraduateStudent)]
        
        if not grad_students:
            print("\nNo graduate students have been added to the system.")
            return

        print("\n" + "-" * 80)
        print(f"{'--- Graduate Student Roster ---':^80}")
        print("-" * 80)
        print(f"{'Student ID':<15} | {'Name':<25} | {'Age':<5} | {'Thesis Topic'}")
        print("-" * 80)
        for s in grad_students:
            print(f"{s.student_id:<15} | {s.name:<25} | {s.age:<5} | {s.thesis_topic}")
        print("-" * 80)

    def show_undergrad_students(self):
        """Filters and displays only the undergraduate students in a formatted table."""
        # Use a list comprehension to find instances of Student but NOT GraduateStudent
        undergrad_students = [s for s in self.students if type(s) is Student]

        if not undergrad_students:
            print("\nNo undergraduate students have been added to the system.")
            return
        
        print("\n" + "-" * 55)
        print(f"{'--- Undergraduate Student Roster ---':^55}")
        print("-" * 55)
        print(f"{'Student ID':<15} | {'Name':<25} | {'Age':<5}")
        print("-" * 55)
        for s in undergrad_students:
            print(f"{s.student_id:<15} | {s.name:<25} | {s.age:<5}")
        print("-" * 55)

    # --- Course Methods ---
    def add_course(self, new_course_object):
        if any(c.course_code == new_course_object.course_code for c in self.courses):
            print(f"\nError: Course with code {new_course_object.course_code} already exists.")
            return
        self.courses.append(new_course_object)

    def show_courses(self):
        if not self.courses:
            print("\nNo Courses have been added yet.")
            return
        for c in self.courses:
            print("\n" + "-"*80)
            print(c)
            print("-"*80)

    # --- Enrollment & Grade Methods ---
    def enroll_student_in_course(self, student_id, course_code):
        student_found = next((s for s in self.students if s.student_id == student_id), None)
        course_found = next((c for c in self.courses if c.course_code == course_code), None)

        if not student_found or not course_found:
            print("\nError: Student ID or Course Code not found.")
            return

        self.enrollment.setdefault(student_id, {})
        if course_code not in self.enrollment[student_id]:
            self.enrollment[student_id][course_code] = {'grade': 'Not Graded yet', 'marks': 'Not Marked yet'}
            print(f"\nSuccess: {student_found.name} enrolled in {course_found.course_name}.")
        else:
            print(f"\nInfo: {student_found.name} is already enrolled in {course_found.course_name}.")

    def process_grade_assignment(self, student_id, course_code):
        """Finds a student and course, then uses polymorphism to calculate and assign a grade."""
        student = next((s for s in self.students if s.student_id == student_id), None)
        course = next((c for c in self.courses if c.course_code == course_code), None)

        if not student or not course:
            print("\nError: Student or Course not found.")
            return

        if student_id not in self.enrollment or course_code not in self.enrollment[student_id]:
            print(f"\nError: Student {student.name} is not enrolled in {course.course_name}.")
            return
        
        # THIS IS THE POLYMORPHIC CALL
        # Python calls the correct method based on the 'student' object's type
        final_marks, final_grade = student.calculate_course_grade(course)

        if final_marks is not None:
            # Update the enrollment record
            self.enrollment[student_id][course_code]['marks'] = final_marks
            self.enrollment[student_id][course_code]['grade'] = final_grade
            
            # Display the result, including the course credit
            print(f"\n✅ Success! Grade '{final_grade}' assigned for course '{course.course_name}' ({course.course_credit} credits).")

   
    def show_courses(self):
        if not self.courses:
            print("\nNo Courses have been added yet.")
            return

        # Define column widths for consistent spacing
        code_width = 15
        name_width = 30
        credit_width = 8
        total_width = code_width + name_width + credit_width + 30

        # Print the main table header
        print("\n" + "=" * total_width)
        print(f"{'--- Course Roster ---':^{total_width}}")
        print("=" * total_width)
        print(f"{'Course Code':<{code_width}} | {'Course Name':<{name_width}} | {'Credits':<{credit_width}} | {'Subjects'}")
        print("-" * total_width)

        # Loop through each course to print its details
        for course in self.courses:
            # Prepare the main course info string for the first line
            course_info_line = (f"{course.course_code:<{code_width}} | "
                                f"{course.course_name:<{name_width}} | "
                                f"{str(course.course_credit):<{credit_width}} |")

            if not course.subjects:
                # If there are no subjects, print the placeholder text
                print(f"{course_info_line} (No subjects assigned)")
            else:
                # If there are subjects, print the first one on the same line
                first_subject = course.subjects[0]
                print(f"{course_info_line} - {first_subject.subject_name} ({first_subject.credits} credits)")
                
                # For any additional subjects, print them on new lines, indented
                # Create a blank prefix to align them correctly
                blank_prefix = (f"{' ' * code_width} | "
                                f"{' ' * name_width} | "
                                f"{' ' * credit_width} |")
                for subject in course.subjects[1:]:
                    print(f"{blank_prefix} - {subject.subject_name} ({subject.credits} credits)")
            
            # Print a separator after each course for clarity
            print("-" * total_width)

        
        

    def show_all_enrollments(self):
        """Displays a clean table of all students and their enrolled courses."""
        if not self.students:
            print("\nThere are no students in the system.")
            return

        # Define column widths for a tidy table
        id_width = 15
        name_width = 25
        total_width = id_width + name_width + 55

        print("\n" + "=" * total_width)
        print(f"{'--- Student Enrollment Roster ---':^{total_width}}")
        print("=" * total_width)
        print(f"{'Student ID':<{id_width}} | {'Student Name':<{name_width}} | {'Enrolled Courses (Codes)'}")
        print("-" * total_width)

        # Loop through all students to ensure everyone is listed
        for student in self.students:
            courses_str = "Not enrolled in any courses."
            # Check if the student has any enrollments recorded
            if student.student_id in self.enrollment and self.enrollment[student.student_id]:
                courses_str = ", ".join(self.enrollment[student.student_id].keys())
            
            print(f"{str(student.student_id):<{id_width}} | {student.name:<{name_width}} | {courses_str}")
        
        print("-" * total_width)

    def show_student_report_card(self, student_id):
        student_found = next((s for s in self.students if s.student_id == student_id), None)
        if not student_found:
            print(f"\nError: Student with ID {student_id} not found.")
            return

        print("\n" + "="*70)
        print(f"{'--- REPORT CARD ---':^70}")
        print("="*70)
        print(f"Student Name: {student_found.name}\t\tStudent ID: {student_id}")
        print("-" * 70)
        print(f"{'Course Code':<15} | {'Course Name':<20} | {'Marks':<7} | {'Grade':<5}")
        print("-" * 70)

        if student_id in self.enrollment:
            for course_code, details in self.enrollment[student_id].items():
                course = next((c for c in self.courses if c.course_code == course_code), None)
                course_name = course.course_name if course else "Unknown"
                print(f"{course_code:<15} | {course_name:<20} | {details['marks']:<7} | {details['grade']:<5}")
        else:
            print("Not enrolled in any courses yet.".center(70))

        gpa = student_found.calculate_gpa(self.enrollment, self.courses)
        print("-" * 70)
        print(f"{'Overall GPA:':>62} {gpa:.2f}")
        print("=" * 70)

    
    # Add this method inside your Gradebook class

    def show_system_overview(self):
        """Displays a combined view of all students and all available courses."""
        print("\n" + "#"*35)
        print("###      System Overview      ###")
        print("#"*35)
        
        # Displaying All Students
        print("\n\n--- Registered Students ---")
        if not self.students:
            print("No students have been added to the system yet.")
        else:
            self.show_student() 

        # Displaying All Courses
        print("\n\n--- Available Courses ---")
        if not self.courses:
            print("No courses have been added to the system yet.")
        else:
            self.show_courses()

    # --- Edit Methods ---

    def edit_student(self, student_id):
        """Finds a student by ID and allows updating their details."""
        student = next((s for s in self.students if s.student_id == student_id), None)
        if not student:
            print("\nError: Student not found.")
            return

        print("\n--- Editing Student Details ---")
        # Edit name (same as before)
        print(f"Current Name: {student.name}")
        new_name = input("Enter new name (or press Enter to keep current): ").title().strip() or student.name

        # Edit age (same as before)
        print(f"Current Age: {student.age}")
        new_age_str = input("Enter new age (or press Enter to keep current): ").strip()
        
        try:
            new_age = int(new_age_str) if new_age_str else student.age
            if new_age <= 0:
                print("Invalid age. Keeping original age.")
                new_age = student.age
        except ValueError:
            print("Invalid input for age. Keeping original age.")
            new_age = student.age
        
        # Update the student's basic details
        student.name = new_name
        student.age = new_age
        
        # --- NEW PART ---
        # Check if the student is a GraduateStudent
        if isinstance(student, GraduateStudent):
            print(f"Current Thesis Topic: {student.thesis_topic}")
            new_topic = input("Enter new thesis topic (or press Enter to keep current): ").title().strip() or student.thesis_topic
            student.thesis_topic = new_topic  # Update the thesis topic
        
        print(f"\n✅ Student {student_id} updated successfully.")

    
    def edit_course(self, course_code):
        """Finds a course by code and allows updating its details."""
        course = next((c for c in self.courses if c.course_code == course_code), None)
        if not course:
            print("\nError: Course not found.")
            return

        print("\n--- Editing Course Details ---")
        print(f"Current Name: {course.course_name}")
        new_name = input("Enter new name (or press Enter to keep current): ").title().strip() or course.course_name

        print(f"Current Total Credits: {course.course_credit}")
        new_credit_str = input("Enter new total credits (or press Enter to keep current): ").strip()

        try:
            new_credit = int(new_credit_str) if new_credit_str else course.course_credit
            # IMPORTANT: Check if new credit value is valid
            current_subject_credits = sum(s.credits for s in course.subjects)
            if new_credit < current_subject_credits:
                print(f"Error: New credit value ({new_credit}) cannot be less than the sum of assigned subject credits ({current_subject_credits}).")
                new_credit = course.course_credit # Revert to original
            elif new_credit <= 0:
                 print("Error: Credits must be a positive number. Keeping original value.")
                 new_credit = course.course_credit # Revert to original
        except ValueError:
            print("Invalid input for credits. Keeping original value.")
            new_credit = course.course_credit

        course.course_name = new_name
        course.course_credit = new_credit
        print(f"\n✅ Course {course_code} updated successfully.")
    
    def unenroll_student(self, student_id, course_code):
        """Removes a student's enrollment from a single course."""
        # Check if the student exists and is enrolled in the specified course
        if student_id in self.enrollment and course_code in self.enrollment[student_id]:
            # Delete the specific course enrollment from the student's record
            del self.enrollment[student_id][course_code]
            print(f"\n✅ Success: Student {student_id} has been un-enrolled from course {course_code}.")
        else:
            # If they aren't enrolled, show an error
            print(f"\nError: Could not perform action. Student {student_id} is not enrolled in '{course_code}'.")

    

    def delete_student(self, student_id):
        """Deletes a student and all their associated enrollments."""
        student_to_delete = next((s for s in self.students if s.student_id == student_id), None)
        if not student_to_delete:
            print("\nError: Student not found.")
            return

        # Confirmation step
        confirm = input(f"Are you sure you want to delete {student_to_delete.name} ({student_id})? "
                        f"This action cannot be undone. (yes/no): ").lower()
        
        if confirm == 'yes':
            # Remove the student from the main list
            self.students = [s for s in self.students if s.student_id != student_id]
            
            # Remove the student's enrollment data if it exists
            if student_id in self.enrollment:
                del self.enrollment[student_id]
                
            print(f"\n✅ Student {student_to_delete.name} and all their enrollments have been deleted.")
        else:
            print("\nDeletion cancelled.")

    

    def delete_course(self, course_code):
        """Deletes a course and removes it from all student enrollments."""
        course_to_delete = next((c for c in self.courses if c.course_code == course_code), None)
        if not course_to_delete:
            print("\nError: Course not found.")
            return

        # Confirmation step
        confirm = input(f"Are you sure you want to delete {course_to_delete.course_name} ({course_code})? "
                        f"This will un-enroll all students from this course. (yes/no): ").lower()

        if confirm == 'yes':
            # Remove the course from the main list
            self.courses = [c for c in self.courses if c.course_code != course_code]

            # Clean up enrollments: iterate through all students and remove the course
            for student_id in list(self.enrollment.keys()):
                if course_code in self.enrollment[student_id]:
                    del self.enrollment[student_id][course_code]
            
            print(f"\n✅ Course {course_to_delete.course_name} has been deleted and all students un-enrolled.")
        else:
            print("\nDeletion cancelled.")




    # ---  Data Persistence Methods ---
    def save_data(self):
        # Save students
        with open(STUDENTS_FILE, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['student_id', 'name', 'age', 'type', 'thesis_topic'])
            for s in self.students:
                row = [s.student_id, s.name, s.age]
                if isinstance(s, GraduateStudent):
                    row.extend(['graduate', s.thesis_topic])
                else:
                    row.extend(['undergraduate', ''])
                writer.writerow(row)

        # Save courses and their subjects
        with open(COURSES_FILE, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['course_code', 'name', 'credit', 'subjects'])
            for c in self.courses:
                subjects_str = ";".join([f"{s.subject_name}:{s.credits}" for s in c.subjects])
                writer.writerow([c.course_code, c.course_name, c.course_credit, subjects_str])

        # Save enrollments
        with open(ENROLLMENTS_FILE, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['student_id', 'course_code', 'marks', 'grade'])
            for sid, courses in self.enrollment.items():
                for code, details in courses.items():
                    writer.writerow([sid, code, details['marks'], details['grade']])
        
        print("\n✅ Data saved successfully to CSV files.")

    def load_data(self):
        if os.path.exists(STUDENTS_FILE):
            with open(STUDENTS_FILE, 'r') as f:
                reader = csv.reader(f)
                next(reader) # Skip header
                for row in reader:
                    student_id, name, age, s_type, thesis = int(row[0]), row[1], int(row[2]), row[3], row[4]
                    if s_type == 'graduate':
                        self.students.append(GraduateStudent(name, age, student_id, thesis))
                    else:
                        self.students.append(Student(name, age, student_id))

        if os.path.exists(COURSES_FILE):
            with open(COURSES_FILE, 'r') as f:
                reader = csv.reader(f)
                next(reader) # Skip header
                for row in reader:
                    code, name, credit, subjects_str = row[0], row[1], int(row[2]), row[3]
                    course = Course(name, code, credit)
                    if subjects_str:
                        for sub in subjects_str.split(';'):
                            s_name, s_credit = sub.split(':')
                            course.add_subject(Subject(s_name, int(s_credit)))
                    self.courses.append(course)
        
        if os.path.exists(ENROLLMENTS_FILE):
            with open(ENROLLMENTS_FILE, 'r') as f:
                reader = csv.reader(f)
                next(reader) # Skip header
                for row in reader:
                    sid, code, marks, grade = int(row[0]), row[1], row[2], row[3]
                    self.enrollment.setdefault(sid, {})[code] = {'marks': marks, 'grade': grade}

        if any([os.path.exists(f) for f in [STUDENTS_FILE, COURSES_FILE, ENROLLMENTS_FILE]]):
            print("✅ Data loaded from existing CSV files.")
        else:
            print("📄 No save files found. Starting a new session.")

# ----- Input Validation & Essential Functions -----
def get_new_student_info():
    s_name = input("Enter Student's name: ").title().strip()
    s_age = 0
    while s_age <= 0:
        try: s_age = int(input(f"Enter {s_name}'s age: "));
        except: print("Invalid age.")
    s_id = 0
    while s_id <= 100:
        try: s_id = int(input(f"Enter Student ID (>100): "));
        except: print("Invalid ID.")
    return s_name, s_age, s_id

def get_new_grad_student_info():
    g_name, g_age, g_id = get_new_student_info()
    thesis_topic = ""
    while not thesis_topic:
        thesis_topic = input("Enter thesis topic: ").strip().title()
    return g_name, g_age, g_id, thesis_topic

def get_new_course_info():
    c_name = input("Enter Course name: ").title().strip()
    c_code = ""
    while not c_code:
        c_code = input("Enter course code (e.g., CS101): ").upper().strip()
    c_credit = 0
    while not (0 < c_credit <= 125):
        try: c_credit = int(input("Enter course credits (1-125): "))
        except: print("Invalid credits.")
    return c_name, c_code, c_credit

# -------- Menu Driven ----------
# Replace your entire main() function with this one

def main():
    manager = Gradebook()
    while True:
        print("""
    ======================================
    ===== Student Management System ======
    ======================================
    --- Add Options ---
    1  → Add Student(s)
    2  → Add Graduate Student(s)
    3  → Add Course(s)
    4  → Add Subject to Course
    
    --- View Options ---
    5  → Show All Students
    6  → Show All UG Students
    7  → Show All Graduate Students
    8  → Show All Courses
    9  → Show All Enrollments
    10 → Show Student's Report Card
    11 → Show System Overview
    
    --- Edit Options ---
    12 → Edit Student Details
    13 → Edit Course Details
    14 → Un-enroll Student from Course
              
    --- Delete Options ---
    15 → Delete a Student
    16 → Delete a Course
    
    --- Grade Options ---
    17 → Enroll Student in Course
    18 → Assign Course Grade
              
    --- System ---
    19 → Save Data
    0  → Exit
    ======================================
        """)
        try:
            option = int(input("Enter an option: "))
            
            # --- Add Options ---
            if option == 1:
                num = int(input("How many undergraduate students to add? "))
                for i in range(num):
                    print(f"\n--- Adding UG Student Details {i + 1} of {num} ---")
                    student_details = get_new_student_info()
                    manager.add_student(Student(*student_details))
                    student_name = student_details[0]
                    print(f"\n✅ {student_name} has been added to the database.")

            elif option == 2:
                num = int(input("How many graduate students to add? "))
                for i in range(num):
                    print(f"\n--- Adding Graduate Student Details {i + 1} of {num} ---")
                    grad_details = get_new_grad_student_info()
                    manager.add_student(GraduateStudent(*grad_details))
                    
                    print(f"✅ Graduate Student {grad_details[0]} has been added.")

            elif option == 3:
                num = int(input("How many courses to add? "))
                for i in range(num):
                    print(f"\n--- Adding Course Details {i + 1} of {num} ---")
                    course_details = get_new_course_info()
                    manager.add_course(Course(*course_details))
                    print(f"✅ {course_details[0]} has been added.\n")

            elif option == 4:
                manager.show_courses()
                ccode = input("Enter Course Code to add subjects to: ").upper().strip()
                course = next((c for c in manager.courses if c.course_code == ccode), None)
                
                if course:
                    # This is the missing while loop that asks for subject details
                    while course.get_remaining_credits() > 0:
                        print(f"\nAdding to '{course.course_name}' | Remaining Credits: {course.get_remaining_credits()}")
                        s_name = input("Enter subject name: ").title().strip()
                        s_credits = int(input(f"Enter credits for {s_name}: "))
                        
                        if course.add_subject(Subject(s_name, s_credits)):
                            # If adding was successful, check if we should continue
                            if course.get_remaining_credits() > 0:
                                if input("Add another subject? (y/n): ").lower() != 'y':
                                    break # Exit the loop if user says no
                            else:
                                print("\nAll course credits have been assigned.")
                                break # Exit the loop if credits are used up
                        else:
                            print("Please try again.") # This runs if credits would be exceeded
                            
                    print(f"\nFinished adding subjects to {course.course_name}.")
                else: 
                    print("\nCourse not found.")

            # --- View Options ---
            elif option == 5: manager.show_student()
            elif option == 6: manager.show_undergrad_students()
            elif option == 7: manager.show_graduate_students()
            elif option == 8: manager.show_courses()
            elif option == 9: manager.show_all_enrollments()
            elif option == 10:
                manager.show_all_enrollments()
                sid = int(input("Enter Student ID for report card: "))
                manager.show_student_report_card(sid)

            elif option == 11: manager.show_system_overview()

            elif option == 12:
                manager.show_student()
                sid = int(input("Enter the Student ID to edit: "))
                manager.edit_student(sid)
            elif option == 13:
                manager.show_courses()
                ccode = input("Enter the Course Code to edit: ").upper().strip()
                manager.edit_course(ccode)
            elif option == 14:
                manager.show_all_enrollments()
                sid = int(input("Enter the Student ID to un-enroll: "))
                ccode = input("Enter the Course Code to remove: ").upper().strip()
                manager.unenroll_student(sid, ccode)

            # --- Delete Option ---
            elif option == 15:
                manager.show_student()
                sid = int(input("Enter the Student ID to DELETE: "))
                manager.delete_student(sid)


            elif option == 16:
                manager.show_courses()
                ccode = input("Enter the Course Code to DELETE: ").upper().strip()
                manager.delete_course(ccode)

            # --- Grade Options ---
            elif option == 17:
                manager.show_system_overview()
                sid = int(input("Enter Student ID to enroll: "))
                ccode = input("Enter Course Code: ").upper().strip()
                manager.enroll_student_in_course(sid, ccode)
            elif option == 18:
                manager.show_all_enrollments()
                sid = int(input("Enter Student ID to assign a grade to: "))
                ccode = input("Enter the Course Code: ").upper().strip()
                manager.process_grade_assignment(sid, ccode)

            # --- System ---
            elif option == 19: manager.save_data()
            elif option == 0:
                if input("Do you want to save before exiting? (y/n): ").lower() == 'y':
                    manager.save_data()
                print("\nExiting system. Goodbye!")
                break
            else:
                print("\nInvalid option. Please try again.")

        except ValueError:
            print("\nInvalid input. Please enter a number.")
        except KeyboardInterrupt:
            print("\nApplication shutting down.")
            break
    
if __name__ == "__main__":
    main()

📄 No save files found. Starting a new session.

    ===== Student Management System ======
    --- Add Options ---
    1  → Add Student(s)
    2  → Add Graduate Student(s)
    3  → Add Course(s)
    4  → Add Subject to Course

    --- View Options ---
    5  → Show All Students
    6  → Show All UG Students
    7  → Show All Graduate Students
    8  → Show All Courses
    9  → Show All Enrollments
    10 → Show Student's Report Card
    11 → Show System Overview

    --- Edit Options ---
    12 → Edit Student Details
    13 → Edit Course Details
    14 → Un-enroll Student from Course

    --- Delete Options ---
    15 → Delete a Student
    16 → Delete a Course

    --- Grade Options ---
    17 → Enroll Student in Course
    18 → Assign Course Grade

    --- System ---
    19 → Save Data
    0  → Exit
        

--- Adding UG Student Details 1 of 1 ---

✅ Rudra has been added to the database.

    ===== Student Management System ======
    --- Add Options ---
    1  → Add Student(s)
   